# Top 5 Quotes dummy model

In [1]:
import pandas as pd
import itertools
import re
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [2]:
file = "../raw_data/quotes - reduced.csv"
quotes = pd.read_csv(file, decimal=",")
quotes.head(3)

,quote,author,category
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak..."
1,You've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love..."
2,You know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s..."


In [3]:
quotes.shape

(156656, 3)

In [4]:
tags_list = quotes.category.to_list()
number_of_tags = []
for i in tags_list:
    number_of_tags.append(len(str(i).split(',')))

In [5]:
quotes['quotes_length'] = quotes['quote'].copy()
for index, row in quotes.iterrows():
    quotes.loc[index, "quotes_length"] = len(str(row['quotes_length']).split(' '))

## Cleaning data: tags are all lowercase, '-' were replaced with ',', duplicates were removed

In [6]:
for index, row in quotes.iterrows():
    quotes.loc[index, "category"] = str(row['category']).lower().replace('-',', ')

In [7]:
quotes['list_tags'] = quotes['category'].copy()
for index,row in quotes.iterrows():
    quotes.loc[index, 'count_tags'] = len(str(row['list_tags']).split(','))
quotes.count_tags.describe()

count    156656.000000
mean          9.747638
std          15.356019
min           1.000000
25%           3.000000
50%           5.000000
75%          11.000000
max         568.000000
Name: count_tags, dtype: float64

In [8]:
for index,row in quotes.iterrows():
    quotes.at[index, 'list_tags'] = str(row['list_tags']).split(',')

In [9]:
for index,row in quotes.iterrows():
    quotes.at[index, 'list_tags'] = str(set(row['list_tags']))

In [10]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
quotes['list_tags'] = quotes['list_tags'].apply(remove_punctuations)

In [11]:
vectorizer = TfidfVectorizer()
tags_vectorized = vectorizer.fit_transform(quotes['list_tags'])
dummy_tags = LatentDirichletAllocation(n_components=5)
lda_vectors = dummy_tags.fit_transform(tags_vectorized)

In [20]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(dummy_tags, vectorizer.fit(quotes['list_tags']))

Topic 0:
[('inspirational', 3766.9947586154517), ('love', 2714.8693686235474), ('quotes', 2709.111722803761), ('philosophy', 2407.224787727965), ('life', 2198.7735886348546), ('wisdom', 1880.7196705170645), ('truth', 1736.2696067750053), ('happiness', 1698.946305480905), ('poetry', 1518.320740310258), ('education', 1460.1817587391179)]
Topic 1:
[('humor', 2646.955823885395), ('death', 1858.1415475457547), ('romance', 1626.3796108969234), ('love', 1624.262078256113), ('funny', 1237.3509863136303), ('marriage', 1233.917295333167), ('war', 969.4698936371075), ('fantasy', 949.4283364269129), ('family', 939.5260317232834), ('life', 896.5161862826399)]
Topic 2:
[('feminism', 1213.6646160477706), ('nature', 1041.756749099252), ('women', 989.0869236029508), ('psychology', 985.6939949500892), ('society', 791.4053693495979), ('human', 477.42576420744086), ('politics', 442.0450801619616), ('peace', 437.41454705725846), ('of', 419.28564123278795), ('the', 414.6269761718359)]
Topic 3:
[('writing', 

In [21]:
lda_vectors[1]

array([0.76351729, 0.06077734, 0.05866088, 0.05839049, 0.05865399])

In [22]:
quotes.iloc[-1] = ['me, Paris, selfie, singing, book','image','image','5','image','1']

In [ ]:
picture_vectorized = vectorizer.transform(quotes.iloc[-1])
lda_vectors_self = dummy_tags.transform(picture_vectorized)
for index,row in quotes.iterrows():
    quotes['topic'] = quotes.quote.copy()

In [ ]:
for index,row in quotes.iterrows():
    quotes.at[index, 'topic'] = int(np.where(lda_vectors[index] == max(lda_vectors[index]))[0])

__Adding the image caption as the last line to our dataset to be able to see the closest neighbourhs__

In [ ]:
quotes.iloc[-1] = ['This is me, I have taken a selfie in Paris.','image','image','10','image','1','?']

In [ ]:
quotes.loc[quotes['author'] == 'image', ['topic']] = np.where(lda_vectors[-1] == max(lda_vectors[-1]))[0]


In [ ]:
quotes.tail()